# Sprint 5 Full Stack and CRUD Review

## Introduction
Our group decided to make a book review site where users can talk about and interact with books while exploring new suggestions for what to read. We chose this for our group project due to our shared interest in reading.
<br>
<br>
My individual feature is a random book generator where users get random suggestions for what to read next base don the genre they selected. When a user inputs a genre, the random suggestion they recieve is displayed with the book's author, title, short summary, and book cover image. I'm currently working on adding a sub-feature where users can add more books (as suggestions) into the random book generator so other users can get a book reccomendation from a bigger database. I haven't connected this "adding book reccomendations" feature to the main database and it has it's own table in backend. 



## Input/Output requests

In [ ]:
#The frontend demo above connects to this section of the bookrec API:
@bookrec_api.route("/add_bookrec", methods=['POST']) # This is the endpoint to add a book to the savebookrec table
def add_book():
    data = request.get_json() # Get the data from the request
    title = data.get('title') 
    author = data.get('author')
    genre = data.get('genre')
    description = data.get('description')
    cover_image_url = data.get('cover_image_url') 

    if not title or not author:
        return jsonify({"error": "Title and author are required"}), 400

    new_book = SaveBookRec( # Create a new book object
        title=title,
        author=author,
        genre=genre,
        description=description,
        cover_image_url=cover_image_url
    )

    db.session.add(new_book) # Add the new book to the savebookrec table
    db.session.commit() # Commit the changes to the database
    
    return jsonify({"message": "Book added successfully"}), 201

### Frontend - API request and response demo
This is demonstration of a user adding a new book reccomendation to my feature through the frontend. This is what the page looks like at first:
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/bookrec_start_page.png">
<br>
The form to add a new book appears when the user clicks the "Add a New Book Reccomendation" button:
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/add_bookrec_empty_form.png">
<br>
This is the filled out form (Each input box is required):
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/filled_bookrec_form.png">
<br>
Once the user has filled out the form and has clicked the "Done" button, a console message appears to confirm that their form has been added to the backend table:
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/bookrec_form_done.png">
<br>
After clicking "Done", the new book appears in the backend table (number 3):
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/backend_bookrec_added.png">
<br>

### Postman - raw API request
This is the GET method of bookrec API being tested through Postman.
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/postman_addbookrec.png"> 
<br>
This is the backend table with the newly added book
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/postman_addbook_backend.png"> 

## List requests

### Formatting responsa data (JSON) from API to DOM
To format response data (JSON) from an API into the DOM, you typically follow these steps:
<ol>
    <li>ake an API Request: Use JavaScript to make a request to the API endpoint.</li>
    <li>Handle the Response: Process the JSON response from the API. </li>
    <li>Update the DOM: Use JavaScript to update the DOM with the data from the response. </li>
</ol>
Here is an example of this method using JavaScript and Python:

In [ ]:
#The provided Python code defines an API endpoint that returns a random book recommendation in JSON format.
@bookrec_api.route('/random_bookrec', methods=['GET'])
def random_bookrec():
    genre = request.args.get('genre')  # Get the 'genre' parameter from the request 
    #print(f"Received genre: {genre}")  # Debug log
    
    while True: # Loop until a book is found 
        book = get_random_bookrec(genre)
        if book:
            return jsonify({
                'title': book.title,
                'author': book.author,
                'description': book.description,
                'image_cover': book.cover_image_url
            })
        else: # Retry if no books are found in the database for the requested genre
            return jsonify({"error": "No books found, retrying in 5 seconds..."}), 404

In [ ]:
%%js 
//The JavaScript code makes a request to the API endpoint and updates the DOM with the response data.
 function displayBook(book) {
        const { title, author, description, image_cover } = book;
        // Update the DOM (Document Object Model) with book details
        document.getElementById("book_title").innerText = title;
        document.getElementById("book_author").innerText = `By: ${author}`;
        document.getElementById("book_description").innerText = description;
        // Book cover display
        document.getElementById("book_cover").src = image_cover;
        document.getElementById("book_cover").style.display = image_cover ? "block" : "none";      
        // Hide the genre selection and show the book details
        document.getElementById("genre_selection").style.display = "none";
        document.getElementById("book_display").style.display = "block";

API Request: The JavaScript code uses the fetch function to make a GET request to the /random_bookrec endpoint with a specified genre.
<br>
Handle the Response: The response is converted to JSON using response.json(). If the response contains an error, it is logged to the console and displayed in the DOM. Otherwise, the book information is extracted from the response.
<br>
Update the DOM: The book information (title, author, description, and cover image) is formatted into HTML and inserted into an element with the ID book-info.

### Queries from the Database (extraction)

A query is a request for data or information from a database. It is used to retrieve, insert, update, or delete data stored in a database. In context of the provided code, queries are used to interact with the database through SQLAlchemy, a third-party library that provides an ORM (Object-Relational Mapping) for Python. We used SQLAlchemy to create a books database which my API interacts with. 

In [ ]:
# Read a single book recommendation by ID
@bookrec_api.route("/get_bookrec/<int:id>", methods=['GET']) # after the get_bookrec/ enter the id number (USE 1 or 2) of the book you want to get
def get_book(id):
    book = SaveBookRec.query.get(id) 
    #This translates to an SQL SELECT query to fetch a book record by its ID from the savebookrec table.

    if not book:
        return jsonify({"error": "Book not found"}), 404

    return jsonify(book.read()), 200

### CRUD Methods ("class")

Create Method: The add_book function is responsible for creating a new book recommendation. It takes in a POST request in the endpoint /add_bookrec with JSON data containing the title, author, genre, summary, and image cover of the book. The function extracts data from the incoming JSON request using request.get_json().It checks if the title and author fields are present. If not, it returns an error response. A new SaveBookRec object is created with the provided data. The new book object is added to the session and committed to the database.
<br>
<br>
Read Method: The get_books function is responsible for reading all book recommendations. It takes in a GET request in the endpoint /get_bookrecs. It queries all records from the SaveBookRec table using SaveBookRec.query.all(). It formats the queried data into JSON using a list comprehension that calls the read method on each book object. The formatted data is returned with a status code of 200.
<br>
<br>
Update Method: The update_book function is responsible for updating an existing book recommendation. It takes in a PUT request in the endpoint /update_bookrec/<id> with JSON data containing the fields to be updated. The function extracts the updated data from the request and retrieves the existing record by its ID. It updates the record's fields with the new data and commits the changes to the database. 
<br>
<br>
Delete Method: The delete_book function is responsible for deleting an existing book recommendation. It takes in a DELETE request in the endpoint /delete_bookrec/<id>. It retrieves the record by its ID and deletes it from the database. The changes are committed to the database. If the record does not exist, it returns an error response.

In [ ]:
#SaveBookRec class to support CRUD operations
class SaveBookRec(db.Model): # Class to save a book recommendation
    __tablename__ = 'savedbookrecs' # name of the table
    id = db.Column(Integer, primary_key=True)
    title = db.Column(String, nullable=False)
    author = db.Column(String, nullable=False)
    genre = db.Column(String)
    description = db.Column(Text)
    cover_image_url = db.Column(String)

    def __init__(self, title, author, genre, description, cover_image_url): # Constructor to initialize the book recommendation
        self.title = title
        self.author = author
        self.genre = genre
        self.description = description
        self.cover_image_url = cover_image_url

    def read(self): # Function to read the book recommendation
        return {
            'id': self.id,
            'title': self.title,
            'author': self.author,
            'genre': self.genre,
            'description': self.description,
            'cover_image_url': self.cover_image_url
        }
    @staticmethod
    def update_book(book_id, data):
        book = SaveBookRec.query.get(book_id)
        if book:
            book.title = data.get('title', book.title)
            book.author = data.get('author', book.author)
            book.genre = data.get('genre', book.genre)
            book.description = data.get('description', book.description)
            book.cover_image_url = data.get('cover_image_url', book.cover_image_url)
            db.session.commit()
            return book
        return None

    @staticmethod
    def delete_book(book_id):
        book = SaveBookRec.query.get(book_id)
        if book:
            db.session.delete(book)
            db.session.commit()
            return True
        return False

In [ ]:

# Endpoint to save a book recommendation (This is what I'm using for the table checkpoint on Thurs/Fri)
@bookrec_api.route("/add_bookrec", methods=['POST']) # This is the endpoint to add a book to the savebookrec table
def add_book():
    data = request.get_json() # Get the data from the request
    title = data.get('title') 
    author = data.get('author')
    genre = data.get('genre')
    description = data.get('description')
    cover_image_url = data.get('cover_image_url') 

    if not title or not author:
        return jsonify({"error": "Title and author are required"}), 400

    new_book = SaveBookRec( # Create a new book object
        title=title,
        author=author,
        genre=genre,
        description=description,
        cover_image_url=cover_image_url
    )

    db.session.add(new_book) # Add the new book to the savebookrec table
    db.session.commit() # Commit the changes to the database
    
    return jsonify({"message": "Book added successfully"}), 201

# Read all book recommendations
@bookrec_api.route("/get_bookrecs", methods=['GET'])
def get_books():
    books = SaveBookRec.query.all()
    return jsonify([book.read() for book in books]), 200

# Update an existing book recommendation by id number assigned to it on the table
@bookrec_api.route("/update_bookrec/<int:id>", methods=['PUT']) # after the update_bookrec/ enter the id number (USE 3 and change the genre to Fantasy) of the book you want to update
def update_book(id):
    data = request.get_json()
    book = SaveBookRec.query.get(id) # Get the book by ID from the savebookrec table

    if not book:
        return jsonify({"error": "Book not found"}), 404

    book.title = data.get('title', book.title)
    book.author = data.get('author', book.author)
    book.genre = data.get('genre', book.genre)
    book.description = data.get('description', book.description)
    book.cover_image_url = data.get('cover_image_url', book.cover_image_url)

    db.session.commit() # Commit the changes to the database
    
    return jsonify({"message": "Book updated successfully"}), 200

# Delete an existing book recommendation by id number assigned to it on the table
@bookrec_api.route("/delete_bookrec/<int:id>", methods=['DELETE']) # after the delete_bookrec/ enter the id number (USE 3) to deleted the added book (A Feast for Crows)
def delete_book(id):
    book = SaveBookRec.query.get(id) # Get the book by ID from the savebookrec table

    if not book:
        return jsonify({"error": "Book not found"}), 404

    db.session.delete(book) # Delete the book from the savebookrec table
    db.session.commit() # Commit the changes to the database
    
    return jsonify({"message": "Book deleted successfully"}), 200

## Algorithmic Code Request

## Call to Algorithm Request